# Next step prediction

In [ ]:
%run -i ../script/requirements.py
%run -i ../script/requirements_dll.py
%matplotlib inline

## Preprocessing

In [ ]:
df = pd.read_csv('../df_completed_order.csv', dtype = {'_CASE_KEY' : str, 'WERKS' : str, 'EBELN' : str, 'EBELP': str, 'ACTIVITY_EN': str, '_SORTING' : int, 'USER_NAME': str})
df['EVENTTIME'] = pd.to_datetime(df['EVENTTIME'])
df = df.rename(columns={'_CASE_KEY' : 'case:concept:name',
                       'ACTIVITY_EN' : 'concept:name',
                       'EVENTTIME' : 'time:timestamp'})
df.loc[df['RKDST'].isnull(), 'RKDST'] = 0
df.loc[df['RKDST'] == 'x', 'RKDST'] = 1
df.loc[df['MAVPR'].isnull(), 'MAVPR'] = 0
df.loc[df['THWMS'].isnull(), 'THWMS'] = 0
df.loc[df['LATEP'].isnull(), 'LATEP'] = 0
df['MAVPR'] = df['MAVPR'].astype(int)
df['THWMS'] = df['THWMS'].astype(int)
df['LATEP'] = df['LATEP'].astype(int)

sc_sorting = StandardScaler()
df['_SORTING'] = sc_sorting.fit_transform(np.array(df['_SORTING']).reshape(-1, 1))
df = df.sort_values(by = ['WERKS', 'EBELN', 'EBELP'])

df = df.loc[df['concept:name'] != 'Late Shipment'].sort_values(by = ['WERKS','EBELN', 'EBELP', 'time:timestamp', '_SORTING'])
df = df.loc[df['concept:name'] != 'Late Delivery'].sort_values(by = ['WERKS','EBELN', 'EBELP', 'time:timestamp', '_SORTING'])
df = df.loc[df['concept:name'] != 'Due Date passed'].sort_values(by = ['WERKS','EBELN', 'EBELP', 'time:timestamp', '_SORTING'])
#df = df.drop(columns=['ELIKZ'])
df = df.reset_index(drop= True)

n_act, onehot_encoder_act = one_hot_column(df['concept:name'])
n_wer, onehot_encoder_wer = one_hot_column(df['WERKS'])
n_nam, onehot_encoder_nam = one_hot_column(df['NAME1'])
n_mag, onehot_encoder_mag = one_hot_column(df['MATKL'])
n_mat, onehot_encoder_mat = one_hot_column(df['MATNR'])

# Time
A = [d.date() for d in df['time:timestamp']]
B = [d.date() for d in df['time:timestamp'].shift(-1)]
B[-1] = B[-2]
df['time:diff'] = np.busday_count(A, B)
df['time:diff'] = df['time:diff'].shift(1)
df.loc[0, 'time:diff'] = 0
idx = df.reset_index().groupby('case:concept:name')['index'].first().values
df.loc[df.index.isin(idx), 'time:diff'] = 0
sc = StandardScaler()
df['time:diff'] = sc.fit_transform(np.array(df['time:diff']).reshape(-1, 1))
df['concept:name'] = df.groupby('case:concept:name')['concept:name'].transform(lambda x: ','.join(x))
df['time:diff'] = df['time:diff'].astype(str)
df['time:diff'] = df.groupby('case:concept:name')['time:diff'].transform(lambda x: ','.join(x))
df['_SORTING'] = df['_SORTING'].astype(str)
df['_SORTING'] = df.groupby('case:concept:name')['_SORTING'].transform(lambda x: ','.join(x))


In [ ]:
df = pd.read_csv('../df_incompleted_order.csv', dtype = {'_CASE_KEY' : str, 'WERKS' : str, 'EBELN' : str, 'EBELP': str, 'ACTIVITY_EN': str, '_SORTING' : int, 'USER_NAME': str})
df['EVENTTIME'] = pd.to_datetime(df['EVENTTIME'])
df = df.rename(columns={'_CASE_KEY' : 'case:concept:name',
                       'ACTIVITY_EN' : 'concept:name',
                       'EVENTTIME' : 'time:timestamp'})
df.loc[df['RKDST'].isnull(), 'RKDST'] = 0
df.loc[df['RKDST'] == 'x', 'RKDST'] = 1
df.loc[df['MAVPR'].isnull(), 'MAVPR'] = 0
df.loc[df['THWMS'].isnull(), 'THWMS'] = 0
df.loc[df['LATEP'].isnull(), 'LATEP'] = 0
df['MAVPR'] = df['MAVPR'].astype(int)
df['THWMS'] = df['THWMS'].astype(int)
df['LATEP'] = df['LATEP'].astype(int)

df['_SORTING'] = sc_sorting.transform(np.array(df['_SORTING']).reshape(-1, 1))
df = df.sort_values(by = ['WERKS', 'EBELN', 'EBELP'])

df = df.loc[df['concept:name'] != 'Late Shipment'].sort_values(by = ['WERKS','EBELN', 'EBELP', 'time:timestamp', '_SORTING'])
df = df.loc[df['concept:name'] != 'Late Delivery'].sort_values(by = ['WERKS','EBELN', 'EBELP', 'time:timestamp', '_SORTING'])
df = df.loc[df['concept:name'] != 'Due Date passed'].sort_values(by = ['WERKS','EBELN', 'EBELP', 'time:timestamp', '_SORTING'])
df = df.reset_index(drop= True)

# Time
A = [d.date() for d in df['time:timestamp']]
B = [d.date() for d in df['time:timestamp'].shift(-1)]
B[-1] = B[-2]
df['time:diff'] = np.busday_count(A, B)
df['time:diff'] = df['time:diff'].shift(1)
df.loc[0, 'time:diff'] = 0
idx = df.reset_index().groupby('case:concept:name')['index'].first().values
df.loc[df.index.isin(idx), 'time:diff'] = 0
df['time:diff'] = sc.transform(np.array(df['time:diff']).reshape(-1, 1))
df['concept:name'] = df.groupby('case:concept:name')['concept:name'].transform(lambda x: ','.join(x))
df['time:diff'] = df['time:diff'].astype(str)
df['time:diff'] = df.groupby('case:concept:name')['time:diff'].transform(lambda x: ','.join(x))
df['_SORTING'] = df['_SORTING'].astype(str)
df['_SORTING'] = df.groupby('case:concept:name')['_SORTING'].transform(lambda x: ','.join(x))

df = df[['case:concept:name', 'concept:name', 'RKDST', 'WERKS', 'time:diff', 'NAME1', 'MATKL', 'MATNR', 'MAVPR', 'THWMS', 'LATES','LATED','LATEP', '_SORTING']].drop_duplicates().reset_index(drop = True)

traces= []
diff_time = []
sorting = []
for q in range(len(df)):
    string = np.array(df.loc[q, 'concept:name'].split(','),dtype = str).reshape(-1,1)
    onehot_encoded = onehot_encoder_act.transform(string)
    traces.append(onehot_encoded)
    diff_time.append(np.array(df.loc[q, 'time:diff'].split(','),dtype = float))
    sorting.append(np.array(df.loc[q, '_SORTING'].split(','),dtype = float))
print('Traccie: ',len(traces))

rda = df['RKDST']
werks_bef = df['WERKS']
name1_bef = df['NAME1']
matkl_bef = df['MATKL']
matnr_bef = df['MATNR']
mavp = df['MAVPR']
twms = df['THWMS']
lates = df['LATES']
lated = df['LATED']
latep = df['LATEP']

werks = onehot_encoder_wer.transform(np.array(werks_bef).reshape(-1, 1))
name1 = onehot_encoder_nam.transform(np.array(name1_bef).reshape(-1, 1))
matkl = onehot_encoder_mag.transform(np.array(matkl_bef).reshape(-1, 1))
matnr = onehot_encoder_mat.transform(np.array(matnr_bef).reshape(-1, 1))

## RDA
for l in range(len(traces)):
    to_app = np.array([[rda[l]] * len(traces[l])]).reshape(len(traces[l]), 1)
    traces[l] = np.append(traces[l], to_app, axis = 1)
# WERKS
for l in range(len(traces)):
    to_app = np.array([[werks[l]] * len(traces[l])]).reshape(len(traces[l]), n_wer)
    traces[l] = np.append(traces[l], to_app, axis = 1)
# NAME1
for l in range(len(traces)):
    to_app = np.array([[name1[l]] * len(traces[l])]).reshape(len(traces[l]), n_nam)
    traces[l] = np.append(traces[l], to_app, axis = 1)
# MATKL
for l in range(len(traces)):
    to_app = np.array([[matkl[l]] * len(traces[l])]).reshape(len(traces[l]), n_mag)
    traces[l] = np.append(traces[l], to_app, axis = 1)
# MATNR
for l in range(len(traces)):
    to_app = np.array([[matnr[l]] * len(traces[l])]).reshape(len(traces[l]), n_mat)
    traces[l] = np.append(traces[l], to_app, axis = 1)
# TIME
for l in range(len(traces)):
    to_app = diff_time[l].reshape(len(traces[l]),1)
    traces[l] = np.append(traces[l], to_app, axis = 1)
# MAVP
for l in range(len(traces)):
    to_app = np.array([[mavp[l]] * len(traces[l])]).reshape(len(traces[l]), 1)
    traces[l] = np.append(traces[l], to_app, axis = 1)
# TWMS
for l in range(len(traces)):
    to_app = np.array([[twms[l]] * len(traces[l])]).reshape(len(traces[l]), 1)
    traces[l] = np.append(traces[l], to_app, axis = 1)
# LATES
for l in range(len(traces)):
    to_app = np.array([[lates[l]] * len(traces[l])]).reshape(len(traces[l]), 1)
    traces[l] = np.append(traces[l], to_app, axis = 1)
# LATED
for l in range(len(traces)):
    to_app = np.array([[lated[l]] * len(traces[l])]).reshape(len(traces[l]), 1)
    traces[l] = np.append(traces[l], to_app, axis = 1)
# LATEP
for l in range(len(traces)):
    to_app = np.array([[latep[l]] * len(traces[l])]).reshape(len(traces[l]), 1)
    traces[l] = np.append(traces[l], to_app, axis = 1)
# # SORTING
# for l in range(len(traces)):
#     to_app = sorting[l].reshape(len(traces[l]),1)
#     traces[l] = np.append(traces[l], to_app, axis = 1)

In [ ]:
# FULL TRACES
n_zero = traces[0].shape[1]

X = traces.copy()

max_trace = 15

for i in range(len(X)):
    num = max_trace - len(X[i])
    for n in range(num):
        X[i] = np.append(X[i], [[0] * n_zero],  axis=0)

X = np.array(X)

In [ ]:
X_rda_train = X[:,0,n_act].reshape(-1,1)
X_wer_train = X[:,0,n_act+1 : n_act+1+n_wer]
X_nam_train = X[:,0,n_act+1+n_wer : n_act+1+n_wer+n_nam]
X_mag_train = X[:,0,n_act+1+n_wer+n_nam : n_act+1+n_wer+n_nam+n_mag]
X_mat_train = X[:,0,n_act+1+n_wer+n_nam+n_mag : n_act+1+n_wer+n_nam+n_mag+n_mat]
X_time_train = X[:,:, n_act+1+n_wer+n_nam+n_mag+n_mat].reshape(-1,max_trace,1)
X_train = X[:,:,:n_act]

class_weights = np.load('class_weights.npy')
loss = weighted_categorical_crossentropy(class_weights)

## Predictions

In [ ]:
model = keras.models.load_model('model/model_1.h5', custom_objects = {'loss' : loss})
model.summary()

In [ ]:
#prova = Model([model.inputs], model.get_layer('dense_93').output)
yhat, yhat_time, yhat_mavp, yhat_twm, yhat_lates, yhat_lated, yhat_latep = model.predict([X_train, X_time_train, X_rda_train, X_mag_train])

In [ ]:

# prova = Model([model.inputs], model.get_layer('dense_97').output)
# yhat_lates = prova.predict([X_train, X_time_train,X_rda_train, X_wer_train, X_nam_train,X_mag_train, X_mat_train])
# yhat_lates = temperature_scaling_bin(yhat_lates, 1)

# prova = Model([model.inputs], model.get_layer('dense_98').output)
# yhat_lated = prova.predict([X_train, X_time_train,X_rda_train, X_wer_train, X_nam_train,X_mag_train, X_mat_train])
# yhat_lated = temperature_scaling_bin(yhat_lated, 1)

# prova = Model([model.inputs], model.get_layer('dense_99').output)
# yhat_latep = prova.predict([X_train, X_time_train,X_rda_train, X_wer_train, X_nam_train,X_mag_train, X_mat_train])
# yhat_latep = temperature_scaling_bin(yhat_latep, 1)

# _, yhat_time, _, _,_,_ = model.predict([X_train, X_time_train,X_rda_train, X_wer_train, X_nam_train,X_mag_train, X_mat_train])
# days = sc.inverse_transform(yhat_time)

# prova = Model([model.inputs], model.get_layer('dense_95').output)
# yhat_mavp = prova.predict([X_train, X_time_train,X_rda_train, X_wer_train, X_nam_train,X_mag_train, X_mat_train])
# yhat_mavp = temperature_scaling_bin(yhat_mavp, 1)

In [ ]:
days = sc.inverse_transform(yhat_time)

In [ ]:
onehot_encoder_act.inverse_transform(np.identity(n_act))[:,0]

In [ ]:
df_anom = pd.DataFrame({'Maverick Purchase': np.round(yhat_mavp * 100, 1).reshape(-1),
                        '3-Way-Mismatch': np.round(yhat_twm * 100, 1).reshape(-1),
                        'Late Shipment': np.round(yhat_lates * 100, 1).reshape(-1),
                        'Late Delivery': np.round(yhat_lated * 100, 1).reshape(-1),
                        'Late Payment': np.round(yhat_latep* 100, 1).reshape(-1),
                        })
# df_lates = pd.DataFrame(np.round(yhat_lates), columns = ['Next activity time (days)'])
df_anom = df_anom.astype(float).round(2)
df_anom['WERKS'] = df['case:concept:name'].str[:4]
df_anom['EBELN'] = df['case:concept:name'].str[5:12]
df_anom['EBELP'] = df['case:concept:name'].str[13:]
df_anom = df_anom[['WERKS', 'EBELN', 'EBELP', 'Maverick Purchase' , '3-Way-Mismatch', 'Late Shipment' ,'Late Delivery', 'Late Payment' ]]

In [ ]:
df_act = pd.DataFrame(np.round(yhat * 100, 1), columns = onehot_encoder_act.inverse_transform(np.identity(n_act))[:,0])
df_time = pd.DataFrame(np.round(days), columns = ['Next activity time (days)'])
df_pred = df_act.join(df_time).astype(float).round(2)
df_pred['WERKS'] = df['case:concept:name'].str[:4]
df_pred['EBELN'] = df['case:concept:name'].str[5:12]
df_pred['EBELP'] = df['case:concept:name'].str[13:]
df_pred = df_pred[['WERKS', 'EBELN', 'EBELP', 'Next activity time (days)', 'Change Price', 'Change Quantity',
       'Purchase Requisition Item: Rejected', 'Purchase Order Item: Rejected', 'Set Payment Block' ]]
df_pred

In [ ]:
data_pool.create_table(df=df_pred, table_name="PRED", drop_if_exists=True, force = True)
data_pool.create_table(df=df_anom, table_name="ANOM", drop_if_exists=True, force = True)